In [1]:
#+.+
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
import yfinance as yf
from tqdm import tqdm

/Users/enzo/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
headlines=pd.read_csv("headlines.csv")
headlines.head()

,Time/Date,Tickers,Headlines
0,2020-08-28 04:46:00,$CYDY,CytoDyn to update investors on regulatory and...
1,2020-08-28 04:41:00,$AAPL $AAPL $TCEHY,Apple terminates Epic Games' developer account
2,2020-08-28 04:38:00,$CCH,Utz Quality Foods and Collier Creek Holdings ...
3,2020-08-28 04:38:00,$LEGN,"Legend Biotech EPS misses by $0.42, misses on..."
4,2020-08-28 04:37:00,$EDEN,Denmark's AAA rating affirmed by S&P as fisca...


In [3]:
headlines['ticker']=headlines['Tickers'].str.strip().str.split().str[0].str.replace('$','',regex=False)
headlines['date']=pd.to_datetime(headlines['Time/Date'])
headlines.head()

,Time/Date,Tickers,Headlines,ticker,date
0,2020-08-28 04:46:00,$CYDY,CytoDyn to update investors on regulatory and...,CYDY,2020-08-28 04:46:00
1,2020-08-28 04:41:00,$AAPL $AAPL $TCEHY,Apple terminates Epic Games' developer account,AAPL,2020-08-28 04:41:00
2,2020-08-28 04:38:00,$CCH,Utz Quality Foods and Collier Creek Holdings ...,CCH,2020-08-28 04:38:00
3,2020-08-28 04:38:00,$LEGN,"Legend Biotech EPS misses by $0.42, misses on...",LEGN,2020-08-28 04:38:00
4,2020-08-28 04:37:00,$EDEN,Denmark's AAA rating affirmed by S&P as fisca...,EDEN,2020-08-28 04:37:00


In [4]:
#function to obtain next trading day's stock returns
def next_day_return(ticker,date,max_days=7): 
    try: #make sure date is datetime object
        date=pd.to_datetime(date)
        end_date=date+pd.Timedelta(days=max_days) #buffer for weekends
        hist=yf.download(ticker,start=date.strftime('%Y-%m-%d'),end=end_date.strftime('%Y-%m-%d'),progress=False) #downloads stock price history from Yahoo finance using yfinance, fetching prices between current date and 2 days later
       
        if hist.empty: #in case no data
            print(f"No data for {ticker} from {date} to {end_date}")
            return None 
        
        hist=hist.sort_index() #ensure index is datetime and sorted
        
        hist_dates=hist.index #extract dates from hist
        headline_date=pd.Timestamp(date.date()) #list of available trading days in downloaded data
        
        available_dates=hist_dates[hist_dates>=headline_date] #filters hist_dates to keep dates after (inclusive) the headline
        
        if len(hist)<2: #not enough data to calculate return
            return None
        
        first_day=available_dates[0] #first trading day on or after headline date
        second_day=available_dates[1] #next trading day on or after headline date
        
        close_first=hist.loc[first_day,'Close'] #closing price on first day
        close_second=hist.loc[second_day,'Close'] #closing price on second day
        
        return (close_second-close_first)/close_first #return ratio (percent increase/100)
    
    except Exception as e: #print error if something goes wrong
        print(f"Error for {ticker} on {date}: {e}")
        return None

In [5]:
#quick check everything is working 
test_sample = headlines.head(5) #test sample

results = []

for idx, row in tqdm(test_sample.iterrows(), total=len(test_sample)):
    ticker = row['ticker']
    date = row['date']
    ret = next_day_return(ticker, date)

    if hasattr(ret, 'values'): #check if return value is a series, if so get scalar value
        ret_val = ret.values[0]
    else:
        ret_val = ret
    
    print(f"{ticker} | {date.date()} -> Return: {ret_val}")


  0%|          | 0/5 [00:00<?, ?it/s]

YF.download() has changed argument auto_adjust default to True


 40%|████      | 2/5 [00:00<00:00,  4.86it/s]

CYDY | 2020-08-28 -> Return: -0.023255860695907783
AAPL | 2020-08-28 -> Return: 0.03391218846132262



1 Failed download:
['CCH']: YFTzMissingError('possibly delisted; no timezone found')
 80%|████████  | 4/5 [00:00<00:00,  4.49it/s]

No data for CCH from 2020-08-28 04:38:00 to 2020-09-04 04:38:00
CCH | 2020-08-28 -> Return: None
LEGN | 2020-08-28 -> Return: 0.0


100%|██████████| 5/5 [00:01<00:00,  4.81it/s]

EDEN | 2020-08-28 -> Return: 0.0005958932399286543


In [6]:
#function to get return of each row of headlines data
def get_return(row):
    ret=next_day_return(row['ticker'],row['date'])
    if hasattr(ret,'values'): #as during check
        ret_val=ret.values[0]
    else: 
        ret_val=ret
    if isinstance(ret_val,(int,float)): #ensure all returns are scalars
        return ret_val
    else: return np.nan

In [7]:
tqdm.pandas()
headlines['return']=headlines.progress_apply(get_return,axis=1)

  0%|          | 0/14088 [00:00<?, ?it/s]
1 Failed download:
['CCH']: YFTzMissingError('possibly delisted; no timezone found')
  0%|          | 4/14088 [00:00<14:54, 15.75it/s]
1 Failed download:
['CIDM']: YFTzMissingError('possibly delisted; no timezone found')
  0%|          | 7/14088 [00:00<14:05, 16.65it/s]

No data for CCH from 2020-08-28 04:38:00 to 2020-09-04 04:38:00
No data for CIDM from 2020-08-28 04:31:00 to 2020-09-04 04:31:00


  0%|          | 18/14088 [00:01<30:04,  7.80it/s]
1 Failed download:
['FB']: YFTzMissingError('possibly delisted; no timezone found')
  0%|          | 19/14088 [00:02<33:03,  7.09it/s]

No data for FB from 2020-08-28 04:15:00 to 2020-09-04 04:15:00


  0%|          | 22/14088 [00:02<28:54,  8.11it/s]
1 Failed download:
['BMCH']: YFTzMissingError('possibly delisted; no timezone found')
  0%|          | 24/14088 [00:02<32:59,  7.11it/s]

...


1 Failed download:
['SEAS']: YFTzMissingError('possibly delisted; no timezone found')
100%|█████████▉| 14082/14088 [42:43<00:00,  7.41it/s]
1 Failed download:
['TEUM']: YFTzMissingError('possibly delisted; no timezone found')
100%|█████████▉| 14083/14088 [42:43<00:00,  7.01it/s]

No data for SEAS from 2020-07-09 08:51:00 to 2020-07-16 08:51:00
No data for TEUM from 2020-07-09 08:50:00 to 2020-07-16 08:50:00


100%|█████████▉| 14084/14088 [42:43<00:00,  7.11it/s]
1 Failed download:
['CCXI']: YFTzMissingError('possibly delisted; no timezone found')
100%|█████████▉| 14086/14088 [42:43<00:00,  6.69it/s]

No data for CCXI from 2020-07-09 08:45:00 to 2020-07-16 08:45:00


100%|██████████| 14088/14088 [42:44<00:00,  5.49it/s]


In [14]:
#Lot of missing values but did it work for most? -> Yes
print("NaNs: ", headlines['return'].isna().sum())
print("Valid returns: ", headlines['return'].notna().sum())
headlines.head(-5)

NaNs:  3284
Valid returns:  10804


,Time/Date,Tickers,Headlines,ticker,date,return
0,2020-08-28 04:46:00,$CYDY,CytoDyn to update investors on regulatory and...,CYDY,2020-08-28 04:46:00,-0.023256
1,2020-08-28 04:41:00,$AAPL $AAPL $TCEHY,Apple terminates Epic Games' developer account,AAPL,2020-08-28 04:41:00,0.033912
2,2020-08-28 04:38:00,$CCH,Utz Quality Foods and Collier Creek Holdings ...,CCH,2020-08-28 04:38:00,NaN
3,2020-08-28 04:38:00,$LEGN,"Legend Biotech EPS misses by $0.42, misses on...",LEGN,2020-08-28 04:38:00,0.000000
4,2020-08-28 04:37:00,$EDEN,Denmark's AAA rating affirmed by S&P as fisca...,EDEN,2020-08-28 04:37:00,0.000596
...,...,...,...,...,...,...
14078,2020-07-09 08:54:00,$ENTA,Enanta Pharmaceuticals advances EDP-514 progr...,ENTA,2020-07-09 08:54:00,0.001566
14079,2020-07-09 08:54:00,$FATE,Fate Therapeutics on go to advance CAR T cand...,FATE,2020-07-09 08:54:00,-0.028650
14080,2020-07-09 08:51:00,$SEAS $FUN $SIX,Six Flags +5% after Janney singles it out as ...,SEAS,2020-07-09 08:51:00,NaN
14081,2020-07-09 08:50:00,$TEUM,Nasdaq grants listing extension to Pareteum,TEUM,2020-07-09 08:50:00,NaN


In [15]:
#save it
headlines.to_csv("headlines_with_returns.csv",index=False)